In [1]:
from new_model import * 

In [2]:
seed = 100
random.seed(seed)
np.random.seed(seed)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(seed)
if device == 'cuda':
    torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
#Data 다운로드 경로지정
batch_size = 128


#Data Process 
transform_train = transforms.Compose([transforms.RandomCrop(32, padding=4),transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]) 

transform_val = transforms.Compose([transforms.ToTensor(), 
                                    transforms.Normalize((0.4914, 0.4822, 0.4465),(0.2023, 0.1994, 0.2010))])

transform_test = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize((0.4914, 0.4822, 0.4465),(0.2023, 0.1994, 0.2010))])

train_CIFAR10 = torchvision.datasets.CIFAR10(root='../data', train=True, download=True, transform=transform_train)

test_CIFAR10 = torchvision.datasets.CIFAR10(root='../data', train=False, download=True, transform=transform_test)

num_train = int(1.0 * len(train_CIFAR10) * 95 / 100)
num_val = len(train_CIFAR10) - num_train
train_CIFAR10, val_CIFAR10 = torch.utils.data.random_split(train_CIFAR10, [num_train, num_val])

train_loader = torch.utils.data.DataLoader(train_CIFAR10, batch_size=batch_size, shuffle=True, num_workers=2) #num_workers는 데이터 로드시 sub process 몇개 쓸거냐 
val_loader = torch.utils.data.DataLoader(val_CIFAR10, batch_size=batch_size,shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_CIFAR10, batch_size=batch_size, shuffle=True, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
#CIFAR 100 : Data 다운로드 경로지정
batch_size = 128

#Data Process
transform_train = transforms.Compose([transforms.RandomCrop(32, padding=4),transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]) 

transform_val = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]) 

transform_test = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]) 

train_CIFAR100 = torchvision.datasets.CIFAR100(root='../data', train=True, download=True, transform=transform_train)

test_CIFAR100 = torchvision.datasets.CIFAR100(root='../data', train=False, download=True, transform=transform_test)

num_train = int(1.0 * len(train_CIFAR100) * 95 / 100)
num_val = len(train_CIFAR100) - num_train
train_CIFAR100, val_CIFAR100 = torch.utils.data.random_split(train_CIFAR100, [num_train, num_val])

train_loader = torch.utils.data.DataLoader(train_CIFAR100, batch_size=batch_size, shuffle=True, num_workers=2) #num_workers는 데이터 로드시 sub process 몇개 쓸거냐 
val_loader = torch.utils.data.DataLoader(val_CIFAR100, batch_size=batch_size,shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_CIFAR100, batch_size=batch_size, shuffle=True, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
def node_prune(model, pruning_ratio):
    for layer in model.modules():
        if isinstance(layer, M_BasicBlock):
            # calculate cutoff value
            nonzero_vals = layer.get_2norm()[0][layer.M_relu1.active != 0]
            cutoff1 = torch.quantile(nonzero_vals, pruning_ratio)

            # set inactive nodes to 0
            layer.M_relu1.active.data[layer.get_2norm()[0] < cutoff1] *= 0
            
            # calculate cutoff value
            nonzero_vals = layer.get_2norm()[1][layer.M_relu2.active != 0]
            cutoff2 = torch.quantile(nonzero_vals, pruning_ratio)

            # set inactive nodes to 0
            layer.M_relu2.active.data[layer.get_2norm()[1] < cutoff2] *= 0

In [5]:
## fine_tuning
def fine_tuning(model, mode, EPOCHS, INITIAL_LR):
    # total number of training epochs
    CHECKPOINT_PATH = "./save_seed100_CIFAR10"

    best_val_acc = 0
    current_learning_rate = INITIAL_LR
    
    remaining_node1 = sum([sum(layer.M_relu1.active) for layer in model.modules() if isinstance(layer, M_BasicBlock)])
    remaining_node2 = sum([sum(layer.M_relu2.active) for layer in model.modules() if isinstance(layer, M_BasicBlock)])
    print(f'최초 node의 갯수 : {remaining_node1, remaining_node2}')    

    print("==> Training starts!")
    start = time.time()
    optimizer = torch.optim.SGD(model.parameters(), lr=INITIAL_LR, momentum=MOMENTUM, weight_decay=REG)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[60, 120, 160], gamma=0.2)
    # scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100, eta_min=0)    
    for i in range(EPOCHS):
        if i % 40 == 0 :
            print(f'{(i//40)+1}번 자르고 fine tuning을 시작합니다.')
            node_prune(model, ratio)
            
            remaining_node1 = sum([sum(layer.M_relu1.active) for layer in model.modules() if isinstance(layer, M_BasicBlock)])
            remaining_node2 = sum([sum(layer.M_relu2.active) for layer in model.modules() if isinstance(layer, M_BasicBlock)])

            print(f'{(i//40)+1}번 pruning 이후 남은 node 는 : {(remaining_node1, remaining_node2)}')
        
        '''
        train loop
        '''            
        model.train()
        
        # this help you compute the training accuracy
        total_examples = 0
        correct_examples = 0

        train_loss = 0 # track training loss if you want

        # Train the model for 1 epoch.
        for batch_idx, (inputs, targets) in enumerate(train_loader):
            inputs, targets = inputs.to(device).float(), targets.to(device).long() #inputs과 targets는 gpu로 계산

            # compute the output and loss
            y_preds = model(inputs)        
            loss = criterion(y_preds,targets)
            train_loss += loss.item()

            # zero the gradient
            optimizer.zero_grad()

            # backpropagation
            loss.backward()

            # apply gradient and update the weights
            optimizer.step()

            # count the number of correctly predicted samples in the current batch
            y_preds_class = torch.argmax(y_preds, dim=1)
            correct_examples += (targets == y_preds_class).sum().item()
            total_examples += targets.size(0)

        scheduler.step()    #스케쥴러 사용해보기
            
        avg_loss_tr = train_loss / len(train_loader) ###
        avg_acc_tr = correct_examples / total_examples ### 

                                                      
        '''
        validation loop
        '''
                                                      
        # switch to eval mode
        model.eval()
        total_examples = 0
        correct_examples = 0
        val_loss = 0 # again, track the validation loss if you want
        
        # disable gradient during validation, which can save GPU memory
        with torch.no_grad():
            for batch_idx, (inputs, targets) in enumerate(val_loader):
                inputs, targets = inputs.to(device).float(), targets.to(device).long()

                # compute the output and loss
                y_preds = model(inputs)        
                loss = criterion(y_preds,targets)
                val_loss += loss.item()

                # count the number of correctly predicted samples in the current batch
                y_preds_class = torch.argmax(y_preds, dim=1)
                correct_examples += (targets == y_preds_class).sum().item()
                total_examples += targets.size(0)

        avg_loss_val = val_loss / len(val_loader)
        avg_acc_val = correct_examples / total_examples

                                                      
        # save the model checkpoint
        if avg_acc_val > best_val_acc:
            best_val_acc = avg_acc_val
            if not os.path.exists(CHECKPOINT_PATH):
                os.makedirs(CHECKPOINT_PATH)

            state = {'state_dict': model.state_dict(),
                     'epoch': i,
                     'lr': current_learning_rate}
            
            torch.save(state, os.path.join(CHECKPOINT_PATH, '{}.pth'.format(mode)))        
    
        if i % 10 == 9 :  
            end = time.time()
            diff_time = round(end - start,2)
            print("Epoch %d:" %(i+1), f"progress time is {diff_time} sec")
            print("Validation loss: %.4f, Validation accuracy: %.4f" % (avg_loss_val, avg_acc_val))
    print(f"==> Optimization finished! Best validation accuracy: {best_val_acc:.4f}")                       

    return model

In [6]:
###8. test_model
def test_model(model):
    model.to(device)
    model.eval()

    total_examples = 0
    correct_examples = 0
    softmax = torch.nn.Softmax(dim=1)

    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            pred = model(inputs)
            total_examples += inputs.shape[0]

            out = softmax(pred)
            out = torch.max(out, 1)

            correct_examples += torch.sum(targets==out[1]).cpu().data.numpy().tolist()

    avg_acc = correct_examples / total_examples
    print("Total examples is {}, correct examples is {}; Test accuracy: {}".format(total_examples, correct_examples, avg_acc))

## 모델지정

In [8]:
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")

MOMENTUM = 0.9

REG = 5e-4

criterion = nn.CrossEntropyLoss()

ratio = 1-0.2**0.1988

basic = 10

In [10]:
for lr in [0.004, 0.02]:
    model = Wide_ResNet(28, basic, 0.3, 10).to(device)        
    ## train
    checkpoint = torch.load(f"../1.Wide_ResNet_model/saved_model/WR_28x{basic}_cifar10.pth")
    model.load_state_dict(checkpoint['state_dict'])

    fine_tuning(model, mode=f"28x{basic}_0.8_multistepLR_lr{lr}", EPOCHS=200, INITIAL_LR=lr)

    ## test
    checkpoint_path = f"./save_seed{seed}_CIFAR10/28x{basic}_0.8_multistepLR_lr{lr}.pth"
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['state_dict'])
    test_model(model)   

| Wide-Resnet 28x10
최초 node의 갯수 : (tensor(4480., device='cuda:3'), tensor(4480., device='cuda:3'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(3252., device='cuda:3'), tensor(3252., device='cuda:3'))
Epoch 10: progress time is 2696.84 sec
Validation loss: 0.0134, Validation accuracy: 0.9972
Epoch 20: progress time is 5388.29 sec
Validation loss: 0.0253, Validation accuracy: 0.9936
Epoch 30: progress time is 8082.03 sec
Validation loss: 0.0168, Validation accuracy: 0.9944
Epoch 40: progress time is 10778.72 sec
Validation loss: 0.0347, Validation accuracy: 0.9896
2번 자르고 fine tuning을 시작합니다.
2번 pruning 이후 남은 node 는 : (tensor(2356., device='cuda:3'), tensor(2356., device='cuda:3'))
Epoch 50: progress time is 13468.64 sec
Validation loss: 0.1074, Validation accuracy: 0.9652
Epoch 60: progress time is 16157.49 sec
Validation loss: 0.1199, Validation accuracy: 0.9604
Epoch 70: progress time is 18846.59 sec
Validation loss: 0.0508, Validation accuracy: 0.9

# cosine scheduler

In [9]:
## fine_tuning
def fine_tuning(model, mode, EPOCHS, INITIAL_LR):
    # total number of training epochs
    CHECKPOINT_PATH = "./save_seed100_CIFAR10"

    best_val_acc = 0
    current_learning_rate = INITIAL_LR
    
    remaining_node1 = sum([sum(layer.M_relu1.active) for layer in model.modules() if isinstance(layer, M_BasicBlock)])
    remaining_node2 = sum([sum(layer.M_relu2.active) for layer in model.modules() if isinstance(layer, M_BasicBlock)])
    print(f'최초 node의 갯수 : {remaining_node1, remaining_node2}')    

    print("==> Training starts!")
    start = time.time()
    optimizer = torch.optim.SGD(model.parameters(), lr=INITIAL_LR, momentum=MOMENTUM, weight_decay=REG)
    # scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[60, 120, 160], gamma=0.2)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100, eta_min=0)    
    for i in range(EPOCHS):
        if i % 40 == 0 :
            print(f'{(i//40)+1}번 자르고 fine tuning을 시작합니다.')
            node_prune(model, ratio)
            
            remaining_node1 = sum([sum(layer.M_relu1.active) for layer in model.modules() if isinstance(layer, M_BasicBlock)])
            remaining_node2 = sum([sum(layer.M_relu2.active) for layer in model.modules() if isinstance(layer, M_BasicBlock)])

            print(f'{(i//40)+1}번 pruning 이후 남은 node 는 : {(remaining_node1, remaining_node2)}')
        
        '''
        train loop
        '''            
        model.train()
        
        # this help you compute the training accuracy
        total_examples = 0
        correct_examples = 0

        train_loss = 0 # track training loss if you want

        # Train the model for 1 epoch.
        for batch_idx, (inputs, targets) in enumerate(train_loader):
            inputs, targets = inputs.to(device).float(), targets.to(device).long() #inputs과 targets는 gpu로 계산

            # compute the output and loss
            y_preds = model(inputs)        
            loss = criterion(y_preds,targets)
            train_loss += loss.item()

            # zero the gradient
            optimizer.zero_grad()

            # backpropagation
            loss.backward()

            # apply gradient and update the weights
            optimizer.step()

            # count the number of correctly predicted samples in the current batch
            y_preds_class = torch.argmax(y_preds, dim=1)
            correct_examples += (targets == y_preds_class).sum().item()
            total_examples += targets.size(0)

        scheduler.step()    #스케쥴러 사용해보기
            
        avg_loss_tr = train_loss / len(train_loader) ###
        avg_acc_tr = correct_examples / total_examples ### 

                                                      
        '''
        validation loop
        '''
                                                      
        # switch to eval mode
        model.eval()
        total_examples = 0
        correct_examples = 0
        val_loss = 0 # again, track the validation loss if you want
        
        # disable gradient during validation, which can save GPU memory
        with torch.no_grad():
            for batch_idx, (inputs, targets) in enumerate(val_loader):
                inputs, targets = inputs.to(device).float(), targets.to(device).long()

                # compute the output and loss
                y_preds = model(inputs)        
                loss = criterion(y_preds,targets)
                val_loss += loss.item()

                # count the number of correctly predicted samples in the current batch
                y_preds_class = torch.argmax(y_preds, dim=1)
                correct_examples += (targets == y_preds_class).sum().item()
                total_examples += targets.size(0)

        avg_loss_val = val_loss / len(val_loader)
        avg_acc_val = correct_examples / total_examples

                                                      
        # save the model checkpoint
        if avg_acc_val > best_val_acc:
            best_val_acc = avg_acc_val
            if not os.path.exists(CHECKPOINT_PATH):
                os.makedirs(CHECKPOINT_PATH)

            state = {'state_dict': model.state_dict(),
                     'epoch': i,
                     'lr': current_learning_rate}
            
            torch.save(state, os.path.join(CHECKPOINT_PATH, '{}.pth'.format(mode)))        
    
        if i % 10 == 9 :  
            end = time.time()
            diff_time = round(end - start,2)
            print("Epoch %d:" %(i+1), f"progress time is {diff_time} sec")
            print("Validation loss: %.4f, Validation accuracy: %.4f" % (avg_loss_val, avg_acc_val))
    print(f"==> Optimization finished! Best validation accuracy: {best_val_acc:.4f}")                       

    return model

In [10]:
for lr in [0.004, 0.02]:
    model = Wide_ResNet(28, basic, 0.3, 10).to(device)        
    ## train
    checkpoint = torch.load(f"../1.Wide_ResNet_model/saved_model/WR_28x{basic}_cifar10.pth")
    model.load_state_dict(checkpoint['state_dict'])

    fine_tuning(model, mode=f"28x{basic}_0.8_cosine_lr{lr}", EPOCHS=200, INITIAL_LR=lr)

    ## test
    checkpoint_path = f"./save_seed{seed}_CIFAR10/28x{basic}_0.8_cosine_lr{lr}.pth"
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['state_dict'])
    test_model(model)   

| Wide-Resnet 28x10
최초 node의 갯수 : (tensor(4480., device='cuda:2'), tensor(4480., device='cuda:2'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(3252., device='cuda:2'), tensor(3252., device='cuda:2'))
Epoch 10: progress time is 2773.08 sec
Validation loss: 0.0198, Validation accuracy: 0.9948
Epoch 20: progress time is 5539.87 sec
Validation loss: 0.0101, Validation accuracy: 0.9972
Epoch 30: progress time is 8304.03 sec
Validation loss: 0.0116, Validation accuracy: 0.9960
Epoch 40: progress time is 11072.22 sec
Validation loss: 0.0177, Validation accuracy: 0.9944
2번 자르고 fine tuning을 시작합니다.
2번 pruning 이후 남은 node 는 : (tensor(2356., device='cuda:2'), tensor(2356., device='cuda:2'))
Epoch 50: progress time is 13834.79 sec
Validation loss: 0.0629, Validation accuracy: 0.9792
Epoch 60: progress time is 16595.99 sec
Validation loss: 0.0547, Validation accuracy: 0.9824
Epoch 70: progress time is 19357.6 sec
Validation loss: 0.0447, Validation accuracy: 0.98

## 결과확인

In [11]:
for lr in [0.004, 0.02]:
    model = Wide_ResNet(28, basic, 0.3, 10).to(device)        
    ## test
    checkpoint_path = f"./save_seed{seed}_CIFAR10/28x{basic}_0.8_multistepLR_lr{lr}.pth"
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['state_dict'])
    test_model(model)   
for lr in [0.004, 0.02]:
    model = Wide_ResNet(28, basic, 0.3, 10).to(device)        
    ## test
    checkpoint_path = f"./save_seed{seed}_CIFAR10/28x{basic}_0.8_cosine_lr{lr}.pth"
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['state_dict'])
    test_model(model)   

| Wide-Resnet 28x10
Total examples is 10000, correct examples is 9515; Test accuracy: 0.9515
| Wide-Resnet 28x10
Total examples is 10000, correct examples is 9564; Test accuracy: 0.9564
| Wide-Resnet 28x10
Total examples is 10000, correct examples is 9561; Test accuracy: 0.9561
| Wide-Resnet 28x10
Total examples is 10000, correct examples is 9559; Test accuracy: 0.9559


In [11]:
##scheduler 실행 : cosineannealingLR
fine_tuning(model, mode = '28x10_0.8_CIFAR10_scheduler_cosine_lr0.1*0.2', EPOCHS = 200)

최초 node의 갯수 : (tensor(3252., device='cuda:1'), tensor(3252., device='cuda:1'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(2356., device='cuda:1'), tensor(2356., device='cuda:1'))
Epoch 10: progress time is 2478.68 sec
Validation loss: 0.2617, Validation accuracy: 0.9108
Epoch 20: progress time is 4955.91 sec
Validation loss: 0.2866, Validation accuracy: 0.9164
Epoch 30: progress time is 7431.12 sec
Validation loss: 0.2915, Validation accuracy: 0.9196
Epoch 40: progress time is 9905.82 sec
Validation loss: 0.3297, Validation accuracy: 0.9140
41번 자르고 fine tuning을 시작합니다.
41번 pruning 이후 남은 node 는 : (tensor(1708., device='cuda:1'), tensor(1708., device='cuda:1'))
Epoch 50: progress time is 12357.39 sec
Validation loss: 0.2436, Validation accuracy: 0.9308
Epoch 60: progress time is 14807.99 sec
Validation loss: 0.1996, Validation accuracy: 0.9428
Epoch 70: progress time is 17254.84 sec
Validation loss: 0.1908, Validation accuracy: 0.9496
Epoch 80: progr

Wide_ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (M_relu): M_relu()
  (layer1): Sequential(
    (0): M_BasicBlock(
      (conv1): Conv2d(16, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu1): M_relu()
      (dropout): Dropout(p=0.3, inplace=False)
      (conv2): Conv2d(160, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu2): M_relu()
      (shortcut): Sequential(
        (0): Conv2d(16, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): M_BasicBlock(
      (conv1): Conv2d(160, 160, kernel_si

In [10]:
##scheduler 실행 : cosineannealingLR
fine_tuning(model, mode = '28x10_0.8_CIFAR10_scheduler_mult_lr0.1*0.2*0.2', EPOCHS = 200)

최초 node의 갯수 : (tensor(4480., device='cuda:1'), tensor(4480., device='cuda:1'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(3252., device='cuda:1'), tensor(3252., device='cuda:1'))
Epoch 10: progress time is 2499.79 sec
Validation loss: 0.1489, Validation accuracy: 0.9588
Epoch 20: progress time is 4998.29 sec
Validation loss: 0.1762, Validation accuracy: 0.9528
Epoch 30: progress time is 7501.47 sec
Validation loss: 0.1696, Validation accuracy: 0.9496
Epoch 40: progress time is 10075.34 sec
Validation loss: 0.1866, Validation accuracy: 0.9524
41번 자르고 fine tuning을 시작합니다.
41번 pruning 이후 남은 node 는 : (tensor(2356., device='cuda:1'), tensor(2356., device='cuda:1'))
Epoch 50: progress time is 12541.16 sec
Validation loss: 0.2398, Validation accuracy: 0.9316
Epoch 60: progress time is 15070.15 sec
Validation loss: 0.2241, Validation accuracy: 0.9348
Epoch 70: progress time is 17543.05 sec
Validation loss: 0.1592, Validation accuracy: 0.9588
Epoch 80: prog

Wide_ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (M_relu): M_relu()
  (layer1): Sequential(
    (0): M_BasicBlock(
      (conv1): Conv2d(16, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu1): M_relu()
      (dropout): Dropout(p=0.3, inplace=False)
      (conv2): Conv2d(160, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu2): M_relu()
      (shortcut): Sequential(
        (0): Conv2d(16, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): M_BasicBlock(
      (conv1): Conv2d(160, 160, kernel_si

In [9]:
##scheduler 실행 : multistepLR
fine_tuning(model, mode = mode, EPOCHS = 200)

최초 node의 갯수 : (tensor(4480., device='cuda:3'), tensor(4480., device='cuda:3'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(3252., device='cuda:3'), tensor(3252., device='cuda:3'))
Epoch 10: progress time is 2620.03 sec
Validation loss: 0.1136, Validation accuracy: 0.9736
Epoch 20: progress time is 5241.29 sec
Validation loss: 0.1016, Validation accuracy: 0.9792
Epoch 30: progress time is 7857.89 sec
Validation loss: 0.1461, Validation accuracy: 0.9636
Epoch 40: progress time is 10477.37 sec
Validation loss: 0.1902, Validation accuracy: 0.9556
2번 자르고 fine tuning을 시작합니다.
2번 pruning 이후 남은 node 는 : (tensor(2356., device='cuda:3'), tensor(2356., device='cuda:3'))
Epoch 50: progress time is 13093.88 sec
Validation loss: 0.7010, Validation accuracy: 0.8104
Epoch 60: progress time is 15715.67 sec
Validation loss: 0.6139, Validation accuracy: 0.8340
Epoch 70: progress time is 18335.23 sec
Validation loss: 0.4811, Validation accuracy: 0.8696
Epoch 80: progre

Wide_ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (M_relu): M_relu()
  (layer1): Sequential(
    (0): M_BasicBlock(
      (conv1): Conv2d(16, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu1): M_relu()
      (dropout): Dropout(p=0.3, inplace=False)
      (conv2): Conv2d(160, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu2): M_relu()
      (shortcut): Sequential(
        (0): Conv2d(16, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): M_BasicBlock(
      (conv1): Conv2d(160, 160, kernel_si

In [12]:
##scheduler 실행 : cosineannealingLR, lr=0.1*0.2*0.2
fine_tuning(model, mode = mode, EPOCHS = 200)

최초 node의 갯수 : (tensor(4480., device='cuda:3'), tensor(4480., device='cuda:3'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(3252., device='cuda:3'), tensor(3252., device='cuda:3'))
Epoch 10: progress time is 2595.28 sec
Validation loss: 0.1058, Validation accuracy: 0.9760
Epoch 20: progress time is 5224.15 sec
Validation loss: 0.1111, Validation accuracy: 0.9792
Epoch 30: progress time is 7852.25 sec
Validation loss: 0.1430, Validation accuracy: 0.9688
Epoch 40: progress time is 10468.09 sec
Validation loss: 0.1145, Validation accuracy: 0.9772
2번 자르고 fine tuning을 시작합니다.
2번 pruning 이후 남은 node 는 : (tensor(2356., device='cuda:3'), tensor(2356., device='cuda:3'))
Epoch 50: progress time is 13049.68 sec
Validation loss: 0.4808, Validation accuracy: 0.8652
Epoch 60: progress time is 15613.49 sec
Validation loss: 0.4036, Validation accuracy: 0.8860
Epoch 70: progress time is 18193.58 sec
Validation loss: 0.4175, Validation accuracy: 0.8880
Epoch 80: progre

Wide_ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (M_relu): M_relu()
  (layer1): Sequential(
    (0): M_BasicBlock(
      (conv1): Conv2d(16, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu1): M_relu()
      (dropout): Dropout(p=0.3, inplace=False)
      (conv2): Conv2d(160, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu2): M_relu()
      (shortcut): Sequential(
        (0): Conv2d(16, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): M_BasicBlock(
      (conv1): Conv2d(160, 160, kernel_si

In [18]:
##scheduler 실행 : cosineannealingLR, lr = 0.1*0.2
fine_tuning(model, mode = mode, EPOCHS = 200)

최초 node의 갯수 : (tensor(4480., device='cuda:3'), tensor(4480., device='cuda:3'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(3252., device='cuda:3'), tensor(3252., device='cuda:3'))
Epoch 10: progress time is 2607.76 sec
Validation loss: 0.7816, Validation accuracy: 0.7748
Epoch 20: progress time is 5206.44 sec
Validation loss: 0.6978, Validation accuracy: 0.8052
Epoch 30: progress time is 7806.79 sec
Validation loss: 0.8532, Validation accuracy: 0.7760
Epoch 40: progress time is 10411.74 sec
Validation loss: 0.7553, Validation accuracy: 0.8052
2번 자르고 fine tuning을 시작합니다.
2번 pruning 이후 남은 node 는 : (tensor(2356., device='cuda:3'), tensor(2356., device='cuda:3'))
Epoch 50: progress time is 12988.85 sec
Validation loss: 0.9377, Validation accuracy: 0.7548
Epoch 60: progress time is 15563.48 sec
Validation loss: 0.7732, Validation accuracy: 0.7972
Epoch 70: progress time is 18132.83 sec
Validation loss: 0.7062, Validation accuracy: 0.8152
Epoch 80: progre

Wide_ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (M_relu): M_relu()
  (layer1): Sequential(
    (0): M_BasicBlock(
      (conv1): Conv2d(16, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu1): M_relu()
      (dropout): Dropout(p=0.3, inplace=False)
      (conv2): Conv2d(160, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu2): M_relu()
      (shortcut): Sequential(
        (0): Conv2d(16, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): M_BasicBlock(
      (conv1): Conv2d(160, 160, kernel_si

In [32]:
##scheduler 실행 : cosineannealingLR, lr = 0.1*0.2*0.2
fine_tuning(model, mode = mode, EPOCHS = 200)

최초 node의 갯수 : (tensor(4480., device='cuda:0'), tensor(4480., device='cuda:0'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(3252., device='cuda:0'), tensor(3252., device='cuda:0'))
Epoch 10: progress time is 2596.18 sec
Validation loss: 0.0167, Validation accuracy: 0.9960
Epoch 20: progress time is 5189.22 sec
Validation loss: 0.0185, Validation accuracy: 0.9952
Epoch 30: progress time is 7788.87 sec
Validation loss: 0.0226, Validation accuracy: 0.9936
Epoch 40: progress time is 10380.87 sec
Validation loss: 0.0229, Validation accuracy: 0.9940
2번 자르고 fine tuning을 시작합니다.
2번 pruning 이후 남은 node 는 : (tensor(2356., device='cuda:0'), tensor(2356., device='cuda:0'))
Epoch 50: progress time is 12972.23 sec
Validation loss: 0.0537, Validation accuracy: 0.9844
Epoch 60: progress time is 15583.05 sec
Validation loss: 0.0478, Validation accuracy: 0.9856
Epoch 70: progress time is 18168.36 sec
Validation loss: 0.0429, Validation accuracy: 0.9852
Epoch 80: progre

Wide_ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (M_relu): M_relu()
  (layer1): Sequential(
    (0): M_BasicBlock(
      (conv1): Conv2d(16, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu1): M_relu()
      (dropout): Dropout(p=0.3, inplace=False)
      (conv2): Conv2d(160, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu2): M_relu()
      (shortcut): Sequential(
        (0): Conv2d(16, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): M_BasicBlock(
      (conv1): Conv2d(160, 160, kernel_si

## seed 10

In [12]:
## cifar100
##scheduler 실행 : cosineannealingLR, lr = 0.1*0.2*0.2
fine_tuning(model, mode = mode, EPOCHS = 200)

최초 node의 갯수 : (tensor(4480., device='cuda:1'), tensor(4480., device='cuda:1'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(3252., device='cuda:1'), tensor(3252., device='cuda:1'))
Epoch 10: progress time is 2690.82 sec
Validation loss: 0.1442, Validation accuracy: 0.9656
Epoch 20: progress time is 5379.31 sec
Validation loss: 0.1399, Validation accuracy: 0.9708
Epoch 30: progress time is 8068.43 sec
Validation loss: 0.1449, Validation accuracy: 0.9732
Epoch 40: progress time is 10757.5 sec
Validation loss: 0.1437, Validation accuracy: 0.9720
2번 자르고 fine tuning을 시작합니다.
2번 pruning 이후 남은 node 는 : (tensor(2356., device='cuda:1'), tensor(2356., device='cuda:1'))
Epoch 50: progress time is 13444.32 sec
Validation loss: 0.5256, Validation accuracy: 0.8688
Epoch 60: progress time is 16127.6 sec
Validation loss: 0.4229, Validation accuracy: 0.8980
Epoch 70: progress time is 18805.88 sec
Validation loss: 0.4375, Validation accuracy: 0.8896
Epoch 80: progress

Wide_ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (M_relu): M_relu()
  (layer1): Sequential(
    (0): M_BasicBlock(
      (conv1): Conv2d(16, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu1): M_relu()
      (dropout): Dropout(p=0.3, inplace=False)
      (conv2): Conv2d(160, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu2): M_relu()
      (shortcut): Sequential(
        (0): Conv2d(16, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): M_BasicBlock(
      (conv1): Conv2d(160, 160, kernel_si

In [9]:
## cifar100
##scheduler 실행 : cosineannealingLR, lr = 0.1*0.2
fine_tuning(model, mode = mode, EPOCHS = 200)

최초 node의 갯수 : (tensor(4480., device='cuda:1'), tensor(4480., device='cuda:1'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(3252., device='cuda:1'), tensor(3252., device='cuda:1'))
Epoch 10: progress time is 2816.52 sec
Validation loss: 0.8322, Validation accuracy: 0.7748
Epoch 20: progress time is 5628.69 sec
Validation loss: 0.9617, Validation accuracy: 0.7424
Epoch 30: progress time is 8439.33 sec
Validation loss: 0.8241, Validation accuracy: 0.7836
Epoch 40: progress time is 11249.75 sec
Validation loss: 0.7671, Validation accuracy: 0.7960
2번 자르고 fine tuning을 시작합니다.
2번 pruning 이후 남은 node 는 : (tensor(2356., device='cuda:1'), tensor(2356., device='cuda:1'))
Epoch 50: progress time is 14050.75 sec
Validation loss: 0.9712, Validation accuracy: 0.7532
Epoch 60: progress time is 16851.54 sec
Validation loss: 0.7995, Validation accuracy: 0.7972
Epoch 70: progress time is 19654.75 sec
Validation loss: 0.7082, Validation accuracy: 0.8208
Epoch 80: progre

Wide_ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (M_relu): M_relu()
  (layer1): Sequential(
    (0): M_BasicBlock(
      (conv1): Conv2d(16, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu1): M_relu()
      (dropout): Dropout(p=0.3, inplace=False)
      (conv2): Conv2d(160, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu2): M_relu()
      (shortcut): Sequential(
        (0): Conv2d(16, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): M_BasicBlock(
      (conv1): Conv2d(160, 160, kernel_si

## 결과 확인

In [12]:
## cifar10
check_prune = torch.load("./saved_tuning/28x10_0.8_CIFAR10_scheduler_cosine_lr0.1*0.2.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

send_email('28x10_0.8_CIFAR10_scheduler_cosine finished')

Total examples is 10000, correct examples is 9489; Test accuracy: 0.9489


In [11]:
## cifar10
check_prune = torch.load("./saved_tuning/28x10_0.8_CIFAR10_scheduler_mult_lr0.1*0.2*0.2.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

send_email('28x10_0.8_CIFAR10_scheduler_cosine finished')

Total examples is 10000, correct examples is 9536; Test accuracy: 0.9536


## CIFAR100 

In [12]:
## cifar100
check_prune = torch.load("./saved_tuning_CIFAR100/28x10_0.8_multistepLR_lr0.1*0.2*0.2.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

# send_email('28x10_0.6_multistepLR_lr0.1*0.2*0.2 finished')

Total examples is 10000, correct examples is 7828; Test accuracy: 0.7828


In [13]:
## cifar100
check_prune = torch.load("./saved_tuning_CIFAR100/28x10_0.8_cosine_lr0.1*0.2*0.2.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

# send_email('28x10_0.6_multistepLR_lr0.1*0.2*0.2 finished')

Total examples is 10000, correct examples is 7817; Test accuracy: 0.7817


In [19]:
## cifar100
check_prune = torch.load("./saved_tuning_CIFAR100/28x10_0.8_cosine_lr0.1*0.2.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

# send_email('28x10_0.6_multistepLR_lr0.1*0.2*0.2 finished')

Total examples is 10000, correct examples is 7805; Test accuracy: 0.7805


## seed10

In [33]:
## cifar10
check_prune = torch.load("./save_seed10_CIFAR10/28x10_0.8_cosine_lr0.1*0.2*0.2.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

# send_email('28x2_multistepLR_lr0.1*0.2*0.2 finished')

Total examples is 10000, correct examples is 9533; Test accuracy: 0.9533


In [13]:
## cifar100
check_prune = torch.load("./save_seed10_CIFAR100/28x10_0.8_cosine_lr0.1*0.2*0.2.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

# send_email('28x2_multistepLR_lr0.1*0.2*0.2 finished')

Total examples is 10000, correct examples is 7804; Test accuracy: 0.7804


In [10]:
## cifar100
check_prune = torch.load("./save_seed10_CIFAR100/28x10_0.8_cosine_lr0.1*0.2.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

# send_email('28x2_multistepLR_lr0.1*0.2*0.2 finished')

Total examples is 10000, correct examples is 7755; Test accuracy: 0.7755


: 